<a href="https://colab.research.google.com/github/NakkaAnkarao8/cancer-detection/blob/main/Bio_ClinicalBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import BertModel, BertTokenizer
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/data/mtsamples.csv')

In [ ]:
data.head(5)

In [ ]:
data['description'][10]

' Morbid obesity.  Laparoscopic Roux-en-Y gastric bypass, antecolic, antegastric with 25-mm EEA anastamosis, esophagogastroduodenoscopy.'

In [ ]:
data['transcription'][10]

'PREOPERATIVE DIAGNOSIS: , Morbid obesity. ,POSTOPERATIVE DIAGNOSIS: , Morbid obesity. ,PROCEDURE:,  Laparoscopic Roux-en-Y gastric bypass, antecolic, antegastric with 25-mm EEA anastamosis, esophagogastroduodenoscopy. ,ANESTHESIA: , General with endotracheal intubation. ,INDICATIONS FOR PROCEDURE: , This is a 50-year-old male who has been overweight for many years and has tried multiple different weight loss diets and programs.  The patient has now begun to have comorbidities related to the obesity.  The patient has attended our bariatric seminar and met with our dietician and psychologist.  The patient has read through our comprehensive handout and understands the risks and benefits of bypass surgery as evidenced by the signing of our consent form.,PROCEDURE IN DETAIL: , The risks and benefits were explained to the patient.  Consent was obtained.  The patient was taken to the operating room and placed supine on the operating room table.  General anesthesia was administered with endot

In [ ]:
data = data[['transcription', 'medical_specialty']]

In [ ]:
data_counts = data['medical_specialty'].value_counts()
data_counts = data_counts[data_counts<=350].index
data = data[~data['medical_specialty'].isin(data_counts)]

In [ ]:
data = data.drop(data[data['transcription'].isna()].index)

In [ ]:
data_categories  = data.groupby(data['medical_specialty'])
i = 1
print('===========Original Categories =======================')
for catName,dataCategory in data_categories:
    print('Cat:'+str(i)+' '+catName + ' : '+ str(len(dataCategory)) )
    i = i+1
print('==================================')

===========Original Categories =======================
Cat:1  Allergy / Immunology : 7
Cat:2  Autopsy : 8
Cat:3  Bariatrics : 18
Cat:4  Cardiovascular / Pulmonary : 372
Cat:5  Chiropractic : 14
Cat:6  Consult - History and Phy. : 516
Cat:7  Cosmetic / Plastic Surgery : 27
Cat:8  Dentistry : 27
Cat:9  Dermatology : 29
Cat:10  Diets and Nutritions : 10
Cat:11  Discharge Summary : 108
Cat:12  ENT - Otolaryngology : 98
Cat:13  Emergency Room Reports : 75
Cat:14  Endocrinology : 19
Cat:15  Gastroenterology : 230
Cat:16  General Medicine : 259
Cat:17  Hematology - Oncology : 90
Cat:18  Hospice - Palliative Care : 6
Cat:19  IME-QME-Work Comp etc. : 16
Cat:20  Lab Medicine - Pathology : 8
Cat:21  Letters : 23
Cat:22  Nephrology : 81
Cat:23  Neurology : 223
Cat:24  Neurosurgery : 94
Cat:25  Obstetrics / Gynecology : 160
Cat:26  Office Notes : 51
Cat:27  Ophthalmology : 83
Cat:28  Orthopedic : 355
Cat:29  Pain Management : 62
Cat:30  Pediatrics - Neonatal : 70
Cat:31  Physical Medicine - Rehab :

In [ ]:
filtered_data_categories = data_categories.filter(lambda x:x.shape[0] > 50)
final_data_categories = filtered_data_categories.groupby(filtered_data_categories['medical_specialty'])
i=1
print('============Reduced Categories ======================')
for catName,dataCategory in final_data_categories:
    print('Cat:'+str(i)+' '+catName + ' : '+ str(len(dataCategory)) )
    i = i+1

print('============ Reduced Categories ======================')

============Reduced Categories ======================
Cat:1  Cardiovascular / Pulmonary : 372
Cat:2  Consult - History and Phy. : 516
Cat:3  Discharge Summary : 108
Cat:4  ENT - Otolaryngology : 98
Cat:5  Emergency Room Reports : 75
Cat:6  Gastroenterology : 230
Cat:7  General Medicine : 259
Cat:8  Hematology - Oncology : 90
Cat:9  Nephrology : 81
Cat:10  Neurology : 223
Cat:11  Neurosurgery : 94
Cat:12  Obstetrics / Gynecology : 160
Cat:13  Office Notes : 51
Cat:14  Ophthalmology : 83
Cat:15  Orthopedic : 355
Cat:16  Pain Management : 62
Cat:17  Pediatrics - Neonatal : 70
Cat:18  Psychiatry / Psychology : 53
Cat:19  Radiology : 273
Cat:20  SOAP / Chart / Progress Notes : 166
Cat:21  Surgery : 1103
Cat:22  Urology : 158
============ Reduced Categories ======================


In [ ]:
filtered_data_categories['medical_specialty'] =filtered_data_categories['medical_specialty'].apply(lambda x:str.strip(x))
mask = filtered_data_categories['medical_specialty'] == 'Surgery'
filtered_data_categories = filtered_data_categories[~mask]
final_data_categories = filtered_data_categories.groupby(filtered_data_categories['medical_specialty'])
mask = filtered_data_categories['medical_specialty'] == 'SOAP / Chart / Progress Notes'
filtered_data_categories = filtered_data_categories[~mask]
mask = filtered_data_categories['medical_specialty'] == 'Office Notes'
filtered_data_categories = filtered_data_categories[~mask]
mask = filtered_data_categories['medical_specialty'] == 'Consult - History and Phy.'
filtered_data_categories = filtered_data_categories[~mask]
mask = filtered_data_categories['medical_specialty'] == 'Emergency Room Reports'
filtered_data_categories = filtered_data_categories[~mask]
mask = filtered_data_categories['medical_specialty'] == 'Discharge Summary'
filtered_data_categories = filtered_data_categories[~mask]
mask = filtered_data_categories['medical_specialty'] == 'Pain Management'
filtered_data_categories = filtered_data_categories[~mask]
mask = filtered_data_categories['medical_specialty'] == 'General Medicine'
filtered_data_categories = filtered_data_categories[~mask]


mask = filtered_data_categories['medical_specialty'] == 'Neurosurgery'
filtered_data_categories.loc[mask, 'medical_specialty'] = 'Neurology'
mask = filtered_data_categories['medical_specialty'] == 'Nephrology'
filtered_data_categories.loc[mask, 'medical_specialty'] = 'Urology'


i=1
print('============Reduced Categories======================')
for catName,dataCategory in final_data_categories:
    print('Cat:'+str(i)+' '+catName + ' : '+ str(len(dataCategory)) )
    i = i+1
print('============Reduced Categories======================')


data = filtered_data_categories[['transcription', 'medical_specialty']]
data = data.drop(data[data['transcription'].isna()].index)
data.shape

============Reduced Categories======================
Cat:1 Cardiovascular / Pulmonary : 372
Cat:2 Consult - History and Phy. : 516
Cat:3 Discharge Summary : 108
Cat:4 ENT - Otolaryngology : 98
Cat:5 Emergency Room Reports : 75
Cat:6 Gastroenterology : 230
Cat:7 General Medicine : 259
Cat:8 Hematology - Oncology : 90
Cat:9 Nephrology : 81
Cat:10 Neurology : 223
Cat:11 Neurosurgery : 94
Cat:12 Obstetrics / Gynecology : 160
Cat:13 Office Notes : 51
Cat:14 Ophthalmology : 83
Cat:15 Orthopedic : 355
Cat:16 Pain Management : 62
Cat:17 Pediatrics - Neonatal : 70
Cat:18 Psychiatry / Psychology : 53
Cat:19 Radiology : 273
Cat:20 SOAP / Chart / Progress Notes : 166
Cat:21 Urology : 158
============Reduced Categories======================


(2324, 2)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['medical_specialty'] = le.fit_transform(data['medical_specialty'])
data.head(5)

,transcription,medical_specialty
3,"2-D M-MODE: , ,1. Left atrial enlargement wit...",0
4,1. The left ventricular cavity size and wall ...,0
7,"2-D ECHOCARDIOGRAM,Multiple views of the heart...",0
9,"DESCRIPTION:,1. Normal cardiac chambers size....",0
11,"2-D STUDY,1. Mild aortic stenosis, widely calc...",0


In [ ]:
data.shape

(2346, 2)

In [ ]:
data.describe

<bound method NDFrame.describe of                                           transcription  medical_specialty
3     2-D M-MODE: , ,1.  Left atrial enlargement wit...                  0
4     1.  The left ventricular cavity size and wall ...                  0
7     2-D ECHOCARDIOGRAM,Multiple views of the heart...                  0
9     DESCRIPTION:,1.  Normal cardiac chambers size....                  0
11    2-D STUDY,1. Mild aortic stenosis, widely calc...                  0
...                                                 ...                ...
4972  INDICATION: , Chest pain.,TYPE OF TEST: , Aden...                  0
4973  CHIEF COMPLAINT: , Chest pain.,HISTORY OF PRES...                  0
4974  HISTORY OF PRESENT ILLNESS: , The patient is a...                  0
4975  HISTORY OF PRESENT ILLNESS: , Mr. ABC is a 60-...                  0
4976  REASON FOR CONSULTATION:  ,Abnormal echocardio...                  0

[2346 rows x 2 columns]>

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2346 entries, 3 to 4976
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   transcription      2330 non-null   object
 1   medical_specialty  2346 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 55.0+ KB


In [ ]:
data['medical_specialty'].value_counts()

3    1103
1     516
0     372
2     355
Name: medical_specialty, dtype: int64

In [ ]:
labels = data['medical_specialty'].tolist()
class_labels = np.unique(labels)
class_weights = len(labels) / (len(class_labels) * np.bincount(labels))
class_weights = torch.tensor(class_weights, dtype=torch.float)
print(class_weights)

tensor([1.5701, 1.1289, 1.6408, 0.5354])


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data['transcription'], data['medical_specialty'],test_size = 0.2, random_state = 1)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model_name = "emilyalsentzer/Bio_ClinicalBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=12)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Define hyperparameters
batch_size = 16
learning_rate = 3e-5
num_epochs = 10

In [ ]:
max_seq_len = 512
text_train = X_train.tolist()
#texts = [(text,) for text in text_train]
tokens_train = tokenizer.batch_encode_plus(
    text_train,
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)
text_test = X_test.tolist()
# tokenize and encode sequences in the validation set
tokens_val = tokenizer.batch_encode_plus(
    text_test,
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False)

In [ ]:
# for train set
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(y_train.tolist())
print("train_y:",train_y)
# for validation set
val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(y_test.tolist())
print("val_y:",val_y)

train_y: tensor([0, 0, 2,  ..., 2, 3, 3])
val_y: tensor([3, 3, 0, 3, 3, 3, 3, 3, 3, 0, 2, 3, 1, 3, 2, 3, 3, 3, 1, 3, 3, 3, 0, 3,
        0, 1, 1, 0, 0, 3, 3, 3, 3, 3, 0, 0, 3, 3, 0, 0, 0, 0, 3, 3, 0, 0, 3, 3,
        3, 3, 3, 2, 3, 0, 1, 3, 3, 3, 1, 1, 0, 2, 3, 0, 3, 3, 3, 0, 3, 3, 2, 3,
        1, 3, 3, 3, 2, 3, 3, 2, 3, 2, 3, 1, 3, 1, 3, 3, 0, 1, 0, 3, 1, 3, 3, 2,
        3, 1, 1, 0, 1, 2, 3, 0, 3, 0, 3, 1, 3, 3, 0, 3, 3, 3, 3, 1, 3, 1, 2, 0,
        1, 3, 3, 3, 1, 2, 2, 1, 2, 2, 3, 0, 1, 0, 2, 2, 3, 1, 1, 2, 0, 3, 3, 3,
        3, 3, 3, 0, 3, 1, 3, 3, 2, 0, 0, 0, 0, 1, 3, 3, 1, 2, 1, 2, 2, 2, 1, 1,
        3, 0, 2, 3, 2, 3, 1, 3, 1, 3, 1, 1, 3, 0, 3, 3, 2, 3, 1, 3, 2, 3, 0, 3,
        3, 3, 1, 1, 2, 0, 3, 3, 0, 2, 3, 2, 3, 0, 3, 3, 1, 3, 3, 0, 2, 0, 1, 3,
        2, 3, 0, 0, 2, 3, 2, 1, 2, 2, 3, 2, 0, 3, 0, 1, 3, 2, 2, 3, 2, 3, 1, 3,
        1, 3, 3, 3, 0, 3, 2, 1, 0, 3, 1, 1, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3,
        1, 3, 3, 3, 3, 1, 2, 3, 3, 0, 3, 2, 0, 1, 3, 2, 2, 2, 3, 3, 0, 

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
train_data = TensorDataset(train_seq, train_mask, train_y)
train_dataloader = DataLoader(train_data, batch_size=batch_size)
test_data = TensorDataset(val_seq, val_mask, val_y)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [ ]:
# Define the optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
# Define the loss function
loss_function = nn.CrossEntropyLoss(weight=class_weights)
# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
model.train()

for epoch in range(num_epochs):
    total_loss = 0
    predictions = []
    true_labels = []

    for batch in train_dataloader:
        sent_id, mask, labels = batch

        optimizer.zero_grad()

        outputs = model(sent_id,mask)

        loss = outputs.loss
        total_loss += loss.item()

        logits = outputs.logits
        _, predicted_labels = torch.max(logits, dim=1)

        predictions.extend(predicted_labels.cpu().tolist())
        true_labels.extend(labels.cpu().tolist())

        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_loss}")
    print(classification_report(true_labels, predictions))

In [ ]:
# Evaluation on test set
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        sent_id, mask, labels = batch
        outputs = model(sent_id, mask)
        logits = outputs.logits
        _, predicted_labels = torch.max(logits, dim=1)

        predictions.extend(predicted_labels.cpu().tolist())
        true_labels.extend(labels.cpu().tolist())

print("Evaluation on Test Set:")
print(classification_report(true_labels, predictions))

# Save the trained model
model.save_pretrained("path/to/save/model")